<a href="https://colab.research.google.com/github/satyamkr1996/ABBSM/blob/master/updated_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import pandas as pd
file = pd.read_csv("/new.csv")

In [93]:
t_ext = 20
t_int = -16

In [94]:
file['Wall_Area'] = file['Wall_1_A'] + file['Wall_2_A'] + file['Wall_3_A']      #outer wall area
file['Window_Area'] = file['Window_1_A'] + file['Window_2_A']              #window area total

In [115]:
def transmission(area, correction_factor, u_value, heat_bridge):
    u_corrected = u_value+heat_bridge
    ht = area*correction_factor*u_corrected
    t_heat_loss = ht*(t_ext-t_int)
    return t_heat_loss

def ventilation(n_min, n50, e, eps, net_vol):      #how net volume?
    v_min = n_min*net_vol
    v_inf = 2*net_vol*n50*e*eps
    vi = max(v_min, v_inf)
    v_heat_loss = 0.34*vi*(t_ext - t_int)
    return v_heat_loss

def heat_up_load(frh, area):
    heatupload = area*frh
    return heatupload

In [125]:
file['transmission_walls']= file.apply(lambda x:transmission(x["Wall_1_A"],1,x["Wall_1_UW"],0.1), axis=1)+file.apply(lambda x:transmission(x["Wall_2_A"],1, x["Wall_2_UW"],0.1), axis=1)+file.apply(lambda x:transmission(x["Wall_3_A"],1, x["Wall_3_UW"],0.1), axis=1)
file['transmission_windows']= file.apply(lambda x:transmission(x["Window_1_A"],1,x["Window_1_UW"],0.1), axis=1)+file.apply(lambda x:transmission(x["Window_2_A"],1, x["Window_2_UW"],0.1), axis=1)
file['transmission_roof']= file.apply(lambda x:transmission(x["Energy_equivalent_Area"],0.4,x["Roof_1_UW"],0.1), axis=1)+file.apply(lambda x:transmission(x["Energy_equivalent_Area"],1, x["Roof_2_UW"],0.1), axis=1)
file['transmission_floor']= file.apply(lambda x:transmission(x["Energy_equivalent_Area"],0.4,x["Floor_1_UW"],0.1), axis=1)+file.apply(lambda x:transmission(x["Energy_equivalent_Area"],1, x["Floor_2_UW"],0.1), axis=1)
file['total_ventilation']=file.apply(lambda x: ventilation(0.5, 3, 0.03, 1, x['Volume']), axis=1)
file['total_heat_up_load']=file.apply(lambda x: heat_up_load(21,x['Wall_1_A'] ), axis=1)+file.apply(lambda x: heat_up_load(21,x['Wall_2_A'] ), axis=1)+file.apply(lambda x: heat_up_load(21,x['Wall_3_A'] ), axis=1)

In [126]:
total_load=['transmission_walls', 'transmission_windows','transmission_roof','transmission_floor','total_ventilation','total_heat_up_load'  ]
file['heat_load']=file[total_load].sum(axis=1)

file['Specific_heat_load'] = file['heat_load']/file['Wall_Area']

In [128]:
file.head()

,House_Type,Year_from,Year_to,number_floors,Energy_equivalent_Area,No_of_residential_building,Volume,Roof_1_A,Roof_2_A,Wall_1_A,...,Wall_Area,Window_Area,transmission_walls,transmission_windows,transmission_roof,transmission_floor,total_ventilation,total_heat_up_load,heat_load,Specific_heat_load
0,EFH_A,0,1859,2,199.0,1,767.575987,134.19,0.0,169.78,...,169.78,28.78,12835.368,3004.632,15617.520,30088.800,4697.565044,3565.38,69809.265044,411.174844
1,EFH_B,1860,1918,2,128.9,1,595.000000,83.12,0.0,194.04,...,194.04,22.30,12573.792,2328.120,7703.064,8445.528,3641.400000,4074.84,38766.744000,199.787384
2,EFH_C,1919,1948,2,275.0,2,1052.500000,213.99,0.0,235.30,...,235.30,52.35,15247.440,5465.340,14850.000,15246.000,6441.300000,4941.30,62191.380000,264.306757
3,EFH_D,1949,1957,1,101.0,1,380.000000,125.40,0.0,117.80,...,117.80,18.40,6361.200,1920.960,5454.000,5650.344,2325.600000,2473.80,24185.904000,205.313277
4,EFH_E,1958,1968,1,110.2,1,502.900000,168.90,0.0,141.20,...,149.90,27.10,6890.040,2829.240,4205.232,9441.936,3077.748000,3147.90,29592.096000,197.412248


In [140]:
file.insert(loc = 0,column = 'PPA',value = '<input type="checkbox" />')
file.style

,PPA,House_Type,Year_from,Year_to,number_floors,Energy_equivalent_Area,No_of_residential_building,Volume,Roof_1_A,Roof_2_A,Wall_1_A,Wall_2_A,Wall_3_A,Floor_1_A,Floor_2_A,Window_1_A,Window_2_A,Door_1_A,Roof_1_UW,Roof_2_UW,Wall_1_UW,Wall_2_UW,Wall_3_UW,Floor_1_UW,Floor_2_UW,Window_1_UW,Window_2_UW,Door_1_UW,Wall_Area,Window_Area,transmission_walls,transmission_windows,transmission_roof,transmission_floor,total_ventilation,total_heat_up_load,heat_load,Specific_heat_load
0,,EFH_A,0,1859,2,199.000000,1,767.575987,134.190000,0.000000,169.780000,0.000000,0,0.000000,85.460000,28.780000,0.000000,2.000000,2.600000,1.000000,2.000000,2.000000,2.000000,2.900000,2.900000,2.800000,5.000000,3.000000,169.780000,28.780000,12835.368000,3004.632000,15617.520000,30088.800000,4697.565044,3565.380000,69809.265044,411.174844
1,,EFH_B,1860,1918,2,128.900000,1,595.000000,83.120000,0.000000,194.040000,0.000000,0,45.600000,32.700000,22.300000,0.000000,2.000000,1.300000,1.000000,1.700000,1.700000,1.700000,1.200000,1.200000,2.800000,2.800000,3.000000,194.040000,22.300000,12573.792000,2328.120000,7703.064000,8445.528000,3641.400000,4074.840000,38766.744000,199.787384
2,,EFH_C,1919,1948,2,275.000000,2,1052.500000,213.990000,0.000000,235.300000,0.000000,0,144.900000,0.000000,52.350000,0.000000,2.000000,1.400000,0.800000,1.700000,1.700000,1.700000,1.000000,1.000000,2.800000,2.800000,3.000000,235.300000,52.350000,15247.440000,5465.340000,14850.000000,15246.000000,6441.300000,4941.300000,62191.380000,264.306757
3,,EFH_D,1949,1957,1,101.000000,1,380.000000,125.400000,0.000000,117.800000,0.000000,0,62.000000,17.900000,18.400000,0.000000,2.000000,1.400000,0.800000,1.400000,1.400000,1.400000,1.010000,1.010000,2.800000,2.800000,3.000000,117.800000,18.400000,6361.200000,1920.960000,5454.000000,5650.344000,2325.600000,2473.800000,24185.904000,205.313277
4,,EFH_E,1958,1968,1,110.200000,1,502.900000,168.900000,0.000000,141.200000,8.700000,0,115.800000,0.000000,27.100000,0.000000,2.100000,0.800000,0.600000,1.200000,0.800000,1.200000,1.600000,1.600000,2.800000,2.800000,3.000000,149.900000,27.100000,6890.040000,2829.240000,4205.232000,9441.936000,3077.748000,3147.900000,29592.096000,197.412248
5,,EFH_F,1969,1978,1,157.500000,1,606.000000,183.130000,0.000000,177.550000,0.000000,0,78.320000,74.000000,34.210000,0.000000,2.000000,0.500000,0.600000,1.000000,1.000000,1.000000,1.000000,1.000000,2.800000,2.800000,3.000000,177.550000,34.210000,7030.980000,3571.524000,5329.800000,8731.800000,3708.720000,3728.550000,32101.374000,180.801881
6,,EFH_G,1979,1983,2,196.000000,1,647.000000,100.800000,0.000000,159.400000,0.000000,0,83.400000,0.000000,27.000000,0.000000,2.000000,0.500000,0.500000,0.800000,0.800000,0.800000,0.800000,0.800000,4.300000,4.300000,3.000000,159.400000,27.000000,5164.560000,4276.800000,5927.040000,8890.560000,3959.640000,3347.400000,31566.000000,198.030113
7,,EFH_H,1984,1994,1,136.550000,1,514.000000,123.200000,0.000000,211.300000,0.000000,0,75.330000,0.000000,29.670000,0.000000,2.000000,0.400000,0.400000,0.500000,0.500000,0.500000,0.600000,0.600000,3.200000,3.200000,3.000000,211.300000,29.670000,4564.080000,3524.796000,3441.060000,4817.484000,3145.680000,4437.300000,23930.400000,113.253195
8,,EFH_I,1995,2001,1,110.800000,1,427.300000,115.500000,0.000000,126.600000,0.000000,0,84.307600,0.000000,32.460000,0.000000,2.000000,0.350000,0.300000,0.300000,0.300000,0.300000,0.450000,0.450000,1.900000,1.900000,2.000000,126.600000,32.460000,1823.040000,2337.120000,2313.504000,3071.376000,2615.076000,2658.600000,14818.716000,117.051469
9,,EFH_J,2002,2009,2,133.200000,1,478.920600,85.910000,0.000000,188.860000,0.000000,0,79.820000,0.000000,28.300000,0.000000,2.000000,0.250000,0.240000,0.300000,0.300000,0.300000,0.300000,0.300000,1.400000,1.400000,2.000000,188.860000,28.300000,2719.584000,1528.200000,2301.696000,2685.312000,2930.994072,3966.060000,16131.846072,85.416955


In [143]:
file.head(2)

,House_Type,Year_from,Year_to,number_floors,Energy_equivalent_Area,No_of_residential_building,Volume,Roof_1_A,Roof_2_A,Wall_1_A,...,Wall_Area,Window_Area,transmission_walls,transmission_windows,transmission_roof,transmission_floor,total_ventilation,total_heat_up_load,heat_load,Specific_heat_load
0,EFH_A,0,1859,2,199.0,1,767.575987,134.19,0.0,169.78,...,169.78,28.78,12835.368,3004.632,15617.520,30088.800,4697.565044,3565.38,69809.265044,411.174844
1,EFH_B,1860,1918,2,128.9,1,595.000000,83.12,0.0,194.04,...,194.04,22.30,12573.792,2328.120,7703.064,8445.528,3641.400000,4074.84,38766.744000,199.787384


In [144]:
file[['House_Type','heat_load' ]]

,House_Type,heat_load
0,EFH_A,6.980927e+04
1,EFH_B,3.876674e+04
2,EFH_C,6.219138e+04
3,EFH_D,2.418590e+04
4,EFH_E,2.959210e+04
5,EFH_F,3.210137e+04
6,EFH_G,3.156600e+04
7,EFH_H,2.393040e+04
8,EFH_I,1.481872e+04
9,EFH_J,1.613185e+04
